<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1/TP1_Parte2_Python_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile tp1-parte2.py
import sys
import os
import random
from threading import Thread
MIN_MATRIX_SIZE = 5
MAX_MATRIX_SIZE = 20
MIN_MATRIX_CELL_VALUE = -32
MAX_MATRIX_CELL_VALUE = 32

def validate_args(args):
    if not len(args) == 2:
        print("Cantidad incorrecta de argumentos")
        os._exit(1)
    size = int(args[1])
    if size > MAX_MATRIX_SIZE or size < MIN_MATRIX_SIZE:
        print("El tamaño debe estar entre {min} y {max}".format(
          min=MIN_MATRIX_SIZE,
          max=MAX_MATRIX_SIZE))
        os._exit(1)
    return size

def matrix_generator(size):
    matrix_result=[]
    for i in range(0, size):
        matrix_row=[]
        for j in range(0, size):
            matrix_row.append(random.randint(
              MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE))
        matrix_result.append(matrix_row)
    return matrix_result

def print_matrix(alias,matrix):
    print("\nMatrix: {alias}".format(alias=alias))
    for row in matrix:
        for col in row:
            print("\t{col}\t".format(col=col),end="")
        print("\n",end="")

def secuential_mult(size, matrix_a, matrix_b):
    matrix_result = []
    for i in range(0, size):
        row_result = [None] * size
        for j in range(0,size):
            for k in range(0,size):
                row_result[j] = matrix_a[i][k] * matrix_b[k][j]
        matrix_result.append(row_result)
    return matrix_result

## Funcion que ejecuta el thread
def multiply_row(size, matrix_a, matrix_b, row_result, rowIndex):
    for j in range(0,size):
        for k in range(0,size):
            row_result[j]= matrix_a[rowIndex][k]*matrix_b[k][j]

## Genera un thread por cada fila encargado de ejecutar la multiplicacion y generar la fila resultante
def concurrent_mult(size, matrix_a, matrix_b):
    matrix_result = []
    threads = []

    for i in range(0, size):
        row_result = [None] * size
        thread = Thread(
          target=multiply_row,
          args=(size, matrix_a, matrix_b, row_result, i))
        thread.start()
        threads.append(thread)
        matrix_result.append(row_result)

    for thread in threads:
        thread.join()

    return matrix_result

def compare_matrix(size, matrix_a, matrix_b):
    for i in range(0, size):
        for j in range(0, size):
            if matrix_a[i][j] != matrix_b[i][j]:
                return False
    return True

def main():
    size = validate_args(sys.argv)
    matrix_a = matrix_generator(size)
    matrix_b = matrix_generator(size)
    matrix_secuential = secuential_mult(size,matrix_a,matrix_b)
    matrix_concurrent = concurrent_mult(size,matrix_a,matrix_b)

    print_matrix("A", matrix_a)
    print_matrix("B", matrix_b)
    print_matrix("Secuencial", matrix_secuential)
    print_matrix("Concurrente", matrix_concurrent)

    ## Determinamos si las matrices son iguales
    result_compare = (
      "iguales" 
      if compare_matrix(size, matrix_secuential, matrix_concurrent)
      else "distintas")
    print("Las matrices concurrente y secuencial son {compare}".format(
      compare=result_compare))

main()

Writing tp1-parte2.py


In [2]:
!python3 tp1-parte2.py 1

El tamaño debe estar entre 5 y 20


In [3]:
!python3 tp1-parte2.py 7


Matrix: A
	-27		-9		-18		-12		-25		5		6	
	1		-22		0		-2		-22		26		-2	
	0		30		-25		5		-3		-3		-29	
	-19		22		26		-14		8		-5		-10	
	10		-2		14		-7		20		18		17	
	-19		-27		-32		13		30		-6		-10	
	26		4		-7		1		1		17		10	

Matrix: B
	-21		-15		7		-4		12		28		-29	
	-15		-18		-28		-18		-27		15		2	
	-6		11		31		-27		0		11		-32	
	0		32		-1		-3		-31		-2		24	
	21		26		-21		11		-23		24		19	
	30		-32		-4		-25		28		19		-8	
	-6		7		-3		-13		22		0		10	

Matrix: Secuencial
	-36		42		-18		-78		132		0		60	
	12		-14		6		26		-44		0		-20	
	174		-203		87		377		-638		0		-290	
	60		-70		30		130		-220		0		-100	
	-102		119		-51		-221		374		0		170	
	60		-70		30		130		-220		0		-100	
	-60		70		-30		-130		220		0		100	

Matrix: Concurrente
	-36		42		-18		-78		132		0		60	
	12		-14		6		26		-44		0		-20	
	174		-203		87		377		-638		0		-290	
	60		-70		30		130		-220		0		-100	
	-102		119		-51		-221		374		0		170	
	60		-70		30		130		-220		0		-100	
	-60		70		-30		-130		220		0		100	
Las matrices concurrente y secuencial son ig